In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import warnings
import os
from feature_selection import get_data
import FinanceDataReader as fdr
import datetime

#%matplotlib inline

warnings.filterwarnings('ignore')

#plt.rcParams['font.family'] = 'NanumGothic'

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


weight_name = os.path.join('tmp_checkpoint.h5')

In [2]:
df, scaler_kospi = get_data()

In [3]:
df

,한국은행_기준금리,KOSPI,KOSDAQ,CD_91일,국고채_10년,회사채_3년_AA,KOSPI_거래대금_일평균,KOSDAQ_거래대금_일평균,KOSPI_주가이익비율_3,기관투자자_순매수,개인_순매수,투자자_예탁금,파생상품거래_예수금_1,RP,위탁매매_미수금,신용융자_잔고_2,거래대금_일평균_CALL_옵션,거래대금_일평균_PUT_옵션,KOSPI_BINARY
0,0.684211,0.150785,0.523719,0.651852,0.905858,0.557803,0.133986,0.128401,0.110236,0.660705,0.172458,0.080661,0.019894,0.000487,0.994870,0.005722,0.212613,0.125723,0.0
1,0.736842,0.138142,0.512937,0.670370,0.843096,0.533237,0.073347,0.090498,0.108993,0.721468,0.211550,0.064424,0.008211,0.001543,1.000000,0.005518,0.204223,0.150988,0.0
2,0.736842,0.132774,0.489471,0.674074,0.872385,0.528902,0.034976,0.068318,0.104849,0.725261,0.208304,0.050511,0.001297,0.000000,0.731455,0.004496,0.101806,0.094471,1.0
3,0.736842,0.159694,0.517151,0.685185,0.878661,0.520231,0.064868,0.077548,0.133858,0.711395,0.185593,0.083090,0.002425,0.003498,0.805738,0.005678,0.187208,0.086090,0.0
4,0.736842,0.114015,0.441979,0.690741,0.826360,0.492775,0.052435,0.050821,0.098632,0.784206,0.261097,0.043003,0.047697,0.003992,0.462610,0.004466,0.194498,0.152607,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.052632,0.854051,0.937060,0.083333,0.240586,0.095376,0.384382,0.681778,0.239536,0.717829,0.320441,0.941931,0.747508,0.968454,0.094060,0.963737,0.113237,0.148818,0.0
190,0.105263,0.795102,0.900527,0.100000,0.232218,0.122832,0.382876,0.795261,0.219644,0.637877,0.159406,0.929911,0.776359,0.948874,0.152658,0.940149,0.100905,0.119428,1.0
191,0.105263,0.857171,0.994048,0.118519,0.196653,0.111272,0.307691,0.701862,0.176129,0.800442,0.000000,0.954787,0.704850,0.864489,0.151241,0.925292,0.169232,0.098858,0.0
192,0.157895,0.716455,0.773606,0.140741,0.259414,0.143064,0.364686,0.576084,0.152093,0.581643,0.374699,1.000000,0.822928,0.935625,0.139390,0.867558,0.156128,0.183374,1.0


In [4]:
window_size = 10

def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size-1])) # -1을 한 이유: 이미 한달 뒤 변동 여부가 KOSPI_BINARY 칼럼에 값으로 들어가 있음
    return np.array(feature_list), np.array(label_list)

In [5]:
target_col = 'KOSPI_BINARY'
feature_cols = list(df.columns)
feature_cols.remove(target_col)
label_cols = [target_col]

train_feature = df[feature_cols]
train_label = df[label_cols]

In [6]:
train_feature, train_label = make_dataset(train_feature, train_label, window_size)
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

In [7]:
x_train.shape, x_valid.shape

((147, 10, 18), (37, 10, 18))

In [8]:
model = Sequential()
model.add(Bidirectional(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu',
               return_sequences=False))
          )
model.add(Dense(1))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=loss, optimizer=optimizer)
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(weight_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

Train on 147 samples, validate on 37 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/200
128/147 [=========================>....] - ETA: 0s - loss: 0.2285
Epoch 00001: val_loss improved from inf to 0.20187, saving model to tmp_checkpoint.h5
147/147 [==============================] - 3s 19ms/sample - loss: 0.2261 - val_loss: 0.2019
Epoch 2/200
128/147 [=========================>....] - ETA: 0s - loss: 0.1886
Epoch 00002: val_loss improved from 0.20187 to 0.16668, saving model to tmp_checkpoint.h5
147/147 [==============================] - 0s 1ms/sample - loss: 0.1848 - val_loss: 0.1667
Epoch 3/200
128/147 [=========================>....] - ETA: 0s - loss: 0.1614
Epoch 00003: val_loss improved from 0.16668 to 0.14600, saving model to tmp_checkpoint.h5
147/147 [==============================] - 0s 769us/sample - loss: 0.1586 - val_loss: 0.1460
Epoch 4/200
112/147 [=====================>........] - ETA: 0s - loss: 0.1416
Epoch 0

# 최근 데이터에 모델 적용 -> 결과값 저장(이 결과가 자산배분 시 활용됨)

In [10]:
df

,한국은행_기준금리,KOSPI,KOSDAQ,CD_91일,국고채_10년,회사채_3년_AA,KOSPI_거래대금_일평균,KOSDAQ_거래대금_일평균,KOSPI_주가이익비율_3,기관투자자_순매수,개인_순매수,투자자_예탁금,파생상품거래_예수금_1,RP,위탁매매_미수금,신용융자_잔고_2,거래대금_일평균_CALL_옵션,거래대금_일평균_PUT_옵션,KOSPI_BINARY
0,0.684211,0.150785,0.523719,0.651852,0.905858,0.557803,0.133986,0.128401,0.110236,0.660705,0.172458,0.080661,0.019894,0.000487,0.994870,0.005722,0.212613,0.125723,0.0
1,0.736842,0.138142,0.512937,0.670370,0.843096,0.533237,0.073347,0.090498,0.108993,0.721468,0.211550,0.064424,0.008211,0.001543,1.000000,0.005518,0.204223,0.150988,0.0
2,0.736842,0.132774,0.489471,0.674074,0.872385,0.528902,0.034976,0.068318,0.104849,0.725261,0.208304,0.050511,0.001297,0.000000,0.731455,0.004496,0.101806,0.094471,1.0
3,0.736842,0.159694,0.517151,0.685185,0.878661,0.520231,0.064868,0.077548,0.133858,0.711395,0.185593,0.083090,0.002425,0.003498,0.805738,0.005678,0.187208,0.086090,0.0
4,0.736842,0.114015,0.441979,0.690741,0.826360,0.492775,0.052435,0.050821,0.098632,0.784206,0.261097,0.043003,0.047697,0.003992,0.462610,0.004466,0.194498,0.152607,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.052632,0.854051,0.937060,0.083333,0.240586,0.095376,0.384382,0.681778,0.239536,0.717829,0.320441,0.941931,0.747508,0.968454,0.094060,0.963737,0.113237,0.148818,0.0
190,0.105263,0.795102,0.900527,0.100000,0.232218,0.122832,0.382876,0.795261,0.219644,0.637877,0.159406,0.929911,0.776359,0.948874,0.152658,0.940149,0.100905,0.119428,1.0
191,0.105263,0.857171,0.994048,0.118519,0.196653,0.111272,0.307691,0.701862,0.176129,0.800442,0.000000,0.954787,0.704850,0.864489,0.151241,0.925292,0.169232,0.098858,0.0
192,0.157895,0.716455,0.773606,0.140741,0.259414,0.143064,0.364686,0.576084,0.152093,0.581643,0.374699,1.000000,0.822928,0.935625,0.139390,0.867558,0.156128,0.183374,1.0


In [11]:
## 최신 데이터셋 구축 ##

# start_date를 넉넉히 20일 전으로 설정
start_date = datetime.date.today() - datetime.timedelta(20)
start_date = datetime.date.isoformat(start_date)

# 코스피 전일 종가 가져오기
kospi = fdr.DataReader('KS11', start_date)

# 가장 최근 일자의 코스피 종가
kospi_yd = kospi.loc[kospi.index[-1], 'Close']

In [23]:
df_recent

,한국은행_기준금리,KOSPI,KOSDAQ,CD_91일,국고채_10년,회사채_3년_AA,KOSPI_거래대금_일평균,KOSDAQ_거래대금_일평균,KOSPI_주가이익비율_3,기관투자자_순매수,개인_순매수,투자자_예탁금,파생상품거래_예수금_1,RP,위탁매매_미수금,신용융자_잔고_2,거래대금_일평균_CALL_옵션,거래대금_일평균_PUT_옵션,KOSPI_BINARY
193,0.157895,0.732501,0.784826,0.161111,0.301255,0.177746,0.350782,0.463611,0.159552,0.624192,0.23824,0.888828,0.768404,0.940553,0.104835,0.835914,0.179183,0.166157,1.0


In [26]:
# df에서 가장 최근 데이터 가져오기
df_recent = df.loc[[df.index[-1]]]
df_recent.drop('KOSPI_BINARY', axis = 1, inplace=True)
df_recent.reset_index(drop=True, inplace=True)

# 코스피 종가 수정
df_recent['KOSPI'] = (df_recent['KOSPI'] - scaler_kospi['min'])/(scaler_kospi['max'] - scaler_kospi['min'])
df_recent

,한국은행_기준금리,KOSPI,KOSDAQ,CD_91일,국고채_10년,회사채_3년_AA,KOSPI_거래대금_일평균,KOSDAQ_거래대금_일평균,KOSPI_주가이익비율_3,기관투자자_순매수,개인_순매수,투자자_예탁금,파생상품거래_예수금_1,RP,위탁매매_미수금,신용융자_잔고_2,거래대금_일평균_CALL_옵션,거래대금_일평균_PUT_옵션
0,0.157895,-0.475588,0.784826,0.161111,0.301255,0.177746,0.350782,0.463611,0.159552,0.624192,0.23824,0.888828,0.768404,0.940553,0.104835,0.835914,0.179183,0.166157


In [33]:
# weight 로딩
model.load_weights(weight_name)

# 예측
pred = model.predict(y_train)

InvalidArgumentError: transpose expects a vector of size 2. But input(1) is a vector of size 3
	 [[{{node sequential/bidirectional/transpose_2}}]]

In [34]:
x_train.size

26460

In [27]:
df_recent_np = df_recent.to_numpy()